# Introduction

In [1]:
import numpy as np
import pandas as pd

In [2]:
import nltk
import string
import spacy
en_core = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize.regexp import regexp_tokenize

In [3]:
train_df=pd.read_csv('./dataset/train_file.csv')


In [4]:
train_df

,Application/Permit Number,Permit Type,Address,Description,Action Type,Work Type,Applicant Name,Application Date,Issue Date,Final Date,Expiration Date,Status,Contractor,Permit and Complaint Status URL,Master Use Permit,Latitude,Longitude,Location,Category
0,6640280,Construction,1924 10TH AVE E,"Alterations to single family residence, (porti...",ADD/ALT,No plan review,"ZHANG, JERRY",2018-01-16T00:00:00,2018-01-16T00:00:00,NaN,2019-07-16T00:00:00,Permit Issued,NaN,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.636732,-122.319828,"{'latitude': '47.63673222', 'human_address': '...",SINGLE FAMILY / DUPLEX
1,6452959,Construction,9434 DELRIDGE WAY SW,Change use permit from restaurant to residenti...,ADD/ALT,Plan Review,"MARTIN, TERRY",2015-02-13T00:00:00,2015-04-16T00:00:00,2016-11-15T00:00:00,2016-10-16T00:00:00,Permit Closed,NaN,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.518298,-122.355555,"{'latitude': '47.51829831', 'human_address': '...",COMMERCIAL
2,6435127,Construction,535 TERRY AVE N,Construct interior alterations to existing off...,ADD/ALT,No plan review,"ALLEXSAHT, REBECCA",2014-09-04T00:00:00,2014-09-04T00:00:00,2015-11-20T00:00:00,2016-03-04T00:00:00,Permit Closed,IA/INTERIOR ARCHITECTS,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.624025,-122.337567,"{'latitude': '47.62402492', 'human_address': '...",COMMERCIAL
3,6620870,Demolition,830 4TH AVE S,need to remove a fire escape that is not part ...,DEMOLITION,Plan Review,"JUNK, FRANK",NaN,NaN,NaN,NaN,Initial Information Collected,NaN,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.595020,-122.328530,"{'latitude': '47.59502036', 'human_address': '...",COMMERCIAL
4,6363778,Construction,805 NE 106TH ST,"Channel modification to Thornton Creek, stream...",ADD/ALT,Plan Review,"DANIEL, MARK",2013-05-02T00:00:00,2013-12-19T00:00:00,2015-05-14T00:00:00,2015-06-19T00:00:00,Permit Closed,SEATTLE PARKS DEPT,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.705686,-122.320134,"{'latitude': '47.70568601', 'human_address': '...",COMMERCIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33534,6093924,Construction,3516 S JUNEAU ST,Interior repairs and modifications to existing...,NaN,Plan Review,"SINGER, BILL",NaN,NaN,NaN,NaN,Initial Information Collected,NaN,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.550138,-122.288036,"{'latitude': '47.55013793', 'human_address': '...",SINGLE FAMILY / DUPLEX
33535,6420980,Construction,1106 M L KING JR WAY S,Construct south duplex (units C&D) with one su...,ADD/ALT,Plan Review,"ZHANG, MOON",2014-06-04T00:00:00,2014-09-10T00:00:00,2015-09-01T00:00:00,2016-03-10T00:00:00,Permit Closed,NaN,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,3017193.0,47.592970,-122.297593,"{'latitude': '47.59297026', 'human_address': '...",SINGLE FAMILY / DUPLEX
33536,6429706,Construction,5806 57TH AVE NE,Establish use as and construction of single fa...,NEW,Plan Review,"GAGNON, DON & KARIN",2014-10-08T00:00:00,2014-12-05T00:00:00,2015-09-09T00:00:00,2016-06-05T00:00:00,Permit Closed,NaN,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.670508,-122.266558,"{'latitude': '47.67050776', 'human_address': '...",SINGLE FAMILY / DUPLEX
33537,6654781,Construction,4216 S KENNY ST,Construction new 1923 s.f. single story wood f...,NEW,Plan Review,"PASTOR, DONALD",NaN,NaN,NaN,NaN,NaN,NaN,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.549138,-122.279919,"{'latitude': '47.54913813', 'human_address': '...",SINGLE FAMILY / DUPLEX


In [5]:
train_df.dtypes

Application/Permit Number            int64
Permit Type                         object
Address                             object
Description                         object
Action Type                         object
Work Type                           object
Applicant Name                      object
Application Date                    object
Issue Date                          object
Final Date                          object
Expiration Date                     object
Status                              object
Contractor                          object
Permit and Complaint Status URL     object
Master Use Permit                  float64
Latitude                           float64
Longitude                          float64
Location                            object
Category                            object
dtype: object

In [6]:
train_df['Category'].nunique()

5

## Textual Data for Training to get accuracy

In [7]:
train_df['Description'] = train_df['Description'].fillna('')


In [8]:
train_df['clean_description']=[' '.join([w for w in x.lower().split() if w not in stop]) for x in train_df['Description'].tolist()]


In [9]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def remove_punctuation(text):
    return text.translate(str.maketrans('', '',string.punctuation))

def lemmatize(text):
    tokens=regexp_tokenize(text,pattern='\s+',gaps=True) #https://towardsdatascience.com/benchmarking-python-nlp-tokenizers-3ac4735100c5
    return ' '.join([lemmatizer.lemmatize(w) for w in tokens])
    

In [10]:
train_df['clean_description']=train_df['clean_description'].apply(lemmatize)
train_df['clean_description']=train_df['clean_description'].apply(remove_punctuation)

print(train_df['clean_description'])
# text_description = text_description.apply(lambda x: " ".join([y.lemma_ for y in en_core(x)]))

0        alteration single family residence portions ma...
1        change use permit restaurant residential const...
2        construct interior alteration existing office ...
3        need remove fire escape part building fire saf...
4        channel modification thornton creek stream res...
                               ...                        
33534       interior repair modification existing building
33535    construct south duplex units cd one surface pa...
33536    establish use construction single family resid...
33537    construction new 1923 sf single story wood fra...
33538    construct repair existing garage accessory exi...
Name: clean_description, Length: 33539, dtype: object


In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.layers import Embedding,SpatialDropout1D,Activation
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.callbacks import EarlyStopping

In [12]:
t = Tokenizer()
t.fit_on_texts(train_df['clean_description'])
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(train_df['clean_description'])
# print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 25
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs.shape)


(33539, 25)


In [13]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./dataset/glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


Loaded 400000 word vectors.


In [14]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

le = preprocessing.LabelEncoder()


MAX_SEQUENCE_LENGTH = 100
t = Tokenizer()
t.fit_on_texts(train_df['clean_description'])
vocab_size = len(t.word_index) + 1
# integer encode the documents
X = t.texts_to_sequences(train_df['clean_description'])
# print(encoded_docs)
# pad documents to a max length of 4 words

X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH , padding='post')
print(X.shape)

Y = pd.get_dummies(train_df['Category']).values
print('Shape of label tensor:', Y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(33539, 100)
Shape of label tensor: (33539, 5)
(30185, 100) (30185, 5)
(3354, 100) (3354, 5)


In [15]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=25, trainable=False)
model.add(e)
# model.add(Flatten())
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='relu'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, Y, epochs=50)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, Y, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 100)           1711800   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 1,829,693
Trainable params: 117,893
Non-trainable params: 1,711,800
_________________________________________________________________
None
Epoch 1/50
1049/1049 [==============================] - 40s 30ms/step - loss: 0.7548 - accuracy: 0.5249
Epoch 2/50
1049/1049 [==============================] - 32s 30ms/step - loss: 0.4023 - accuracy: 0.5910
Epoch 3/50
1049/1049 [==============================] - 32s 31ms/step - loss: 0.4224 - accuracy: 0.6755
Epoch 4/50
1049/1049 [==============================]

In [ ]:
# MAX_NB_WORDS = 50000
# # Max number of words in each complaint.
# MAX_SEQUENCE_LENGTH = 250
# # This is fixed.
# EMBEDDING_DIM = 100
# model = Sequential()
# # model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
# model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=X.shape[1], trainable=False))
# model.add(SpatialDropout1D(0.2))
# model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(5, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()
# # epochs = 5
# # batch_size = 64

 # history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


In [ ]:
# # train_df[['Latitude','Longitude']].dropna()
# from sklearn.cluster import KMeans ,AgglomerativeClustering

# # creates 5 clusters using hierarchical clustering.
# agc = AgglomerativeClustering(n_clusters =5, affinity='euclidean', linkage='ward')
# train_df['pickup cluster'] = agc.fit_predict(train_df[['Latitude','Longitude']].dropna())

# # creates 5 clusters using k-means clustering algorithm.
# kmeans = KMeans(5)
# clusters = kmeans.fit_predict(train_df[['Latitude','Longitude']])
# train['pickup cluster'] = kmeans.predict(train_df[['Latitude','Longitude']])